<div style="display:block">
    <div style="width: 20%; display: inline-block; text-align: left;">
        <img src="http://upload.wikimedia.org/wikipedia/en/0/0c/Mu_Sigma_Logo.jpg" style="height:75px; margin-left:0px" />
    </div>
    <div style="width: 59%; display: inline-block">
        <h1  style="text-align: center"> Social Media Analytics (SMA)- Treatment pattern Identification </h1>
        <div style="width: 100%; text-align: center; display: inline-block;"><i>Author:</i> <strong>Sruthi S, Varun Behl</strong> </div>
    </div>
    <div style="width: 20%; text-align: right; display: inline-block;">
        <div style="width: 100%; text-align: left; display: inline-block;">
            <i>Created: </i>
            <time datetime="2018-02-14" pubdate>13th Feb 2018</time>
            <i>Modified: </i>
            <time datetime="2018-02-21" pubdate>21st Feb 2018</time>
        </div>
    </div>
</div>

# Objective

The aim of this notebook is to identify if a sentence contains treatment pattern or not. The data for this analysis is scraped from different forums related to melanoma. The scraped data consists of comments of patients (suffering or cured) from melanoma, their friends and family.

### Phase 1:
The first step is to predict whether the post discusses systemic therapy or not. This is done with a SVM clasification algorithm, which is most appropriate for text classification. 


In [42]:
# Importing necassary packages 
import string
import pandas as pd
import numpy as np
import nltk
import re

# Sklearn package to import necessary vectorizing algorithms
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer

#NLTK package for text mining operations 
from nltk.corpus import stopwords as sw
from nltk.corpus import wordnet as wn
from nltk import wordpunct_tokenize
from nltk import WordNetLemmatizer
from nltk import sent_tokenize
from nltk.tokenize import sent_tokenize
from nltk import pos_tag

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report as clsr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
import pickle
from sklearn.cross_validation import KFold
from nltk.tokenize import sent_tokenize
from __future__ import division
from scipy import sparse , vstack , hstack
from sklearn.svm import SVC
from sklearn.utils import check_random_state, safe_indexing

### Step 1 : Importing social media dataset

##### Step 1a : Data scraped and tokenized to sentence level
The imported data set consists of posts which have been tokenized to sentence level with a corresponding column having flags 1 and 0

1 (talking about drug consumption)<br>
0 (no mention of drug or treatment)

In [2]:
# Reading the sentenced tokenized dataset
posts = pd.read_csv('SMA_train_dataset_updated.csv')

In [3]:

posts.head(5)


Tokenized.Sentences           date  Tag
0  I finished my courses (x4) of Ipilimumab, went...       2/8/2013    1
1  I have really struggled with the side effects ...     12/24/2012    1
2  I am just so stresses out trying to decide if ...      7/24/2011    1
3  My Ipilimumab treatment was before the stuff a...  5/5/2016 0:00    1
4  If it is decided that the combination is bette...  2/9/2016 0:00    0

### Step 2 :  Pre-processing text columns

Textual data has to be processed before incorporating it into the model. Various text mining operations are done on the textual data - like cleaning html tags, special characters, stripping URL tags, etc. Word tokens are created at the end of this process, which will be the independent features for the model. 


In [6]:
## Pre-processing the text column

#lemmatizer = WordNetLemmatizer()

def identity(arg):
    """
    Simple identity function works as a passthrough.
    """
    return arg

## Removing HTML tags
def cleanhtml(raw_html):
    cleanr = re.compile(r'<[^>]+>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

## Removing URL
def strip_URL(text):
    return re.sub(r'(https|http)?:\/\/(\w|\.|-|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE)

## Removing Hashtags and '@' tags 
def strip_all_tags(text):
    entity_prefixes = ['@','#','[]']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

## Removing special characters 
def strip_special_chars(text):
    return re.sub('\W+',' ', text)

## Removing extra white spaces
def strip_extra_white_spaces(text):
    return re.sub(' +',' ',text).strip()

## Removing extra new lines
def strip_extra_new_lines(text):
    return re.sub("\n+","\n", text)

def inverse_transform(l):
        sent = ''
        for word in l:
            sent += word+" "
        return [sent]


def tokenize(document):
    document = cleanhtml(document)
    document = strip_URL(document)
    document = strip_all_tags(document)
    document = strip_special_chars(document)
    document = strip_extra_white_spaces(document)
    document = strip_extra_new_lines(document)
    stopwords_custom = pd.read_csv("stopwords1.csv")
    stopwords_custom = stopwords_custom.iloc[:,0].map(lambda x:x.lower())
    document = re.sub(r'\.+','.',document) # replace multiple dots by one dot
    document = re.sub(r'\(|\)','',document) # remove brackets but not its contents
        
    #Breaking the document into sentences
    for sent in sent_tokenize(document):
        # Breaking the sentence into parts of speech-tagged tokens
        for token, tag in pos_tag(wordpunct_tokenize(sent)):
            # Applying pre-processing to the token
            token = token.lower() 

            #If there is a stopword, ignore the token and continue
            if token.lower in stopwords_custom.tolist():
                continue

            # If there is a punctuation, ignore the token and continue
            if all(char in string.punctuation for char in token):
                  continue
            
            yield token

#Lemmatize the token and yield
            #lemma = lemmatize(token, tag)
            #yield lemma

def lemmatize(token, tag):
    tag = {
       'N': wn.NOUN,
       'V': wn.VERB,
       'R': wn.ADV,
       'J': wn.ADJ
    }.get(tag[0], wn.NOUN)

    #return lemmatizer.lemmatize(token, tag)

# Removes words which are repeating consecutively.
def remove_repeats(tokens):
    no_repeat = []
    for word in tokens:
        if (len(no_repeat)!= 0 and word!=no_repeat[-1]) or (len(no_repeat)==0):
            no_repeat.append(word)
    return no_repeat
    
def preprocessing(X):
    tokens_list = []
    no_repeat = []
    count = 0
    for doc in X:
        tokens = list(tokenize(doc))
        tokens_norepeat = remove_repeats(tokens)
        tokens_list.append(tokens_norepeat)
    return tokens_list


#### Step 2a : Pre-processing and creating a list of word tokens

In [7]:
#pre-processed data :List of words (non-string)
words= preprocessing(posts['Tokenized.Sentences'])

In [8]:
# Word Tokens are created after preprocessing of the textual data
print words[0]
print type(words)

['i', 'finished', 'my', 'courses', 'x4', 'of', 'ipilimumab', 'went', 'off', 'pembrolizumab', 'and', 'unfortunately', 'have', 'had', 'signifcant', 'growth']
<type 'list'>


#### Step 2b : Creating a list of strings
For further operation with tfidf, input should be a string of words/phrase. The following code will create strings of phrases which can be appended to a list


In [9]:
#Convert list of words into string
def concatenate_string(list_string):
    string_list = []
    for i in list_string:
        a = " ".join(str(x) for x in i)
        string_list.append(a)
    return string_list

In [10]:
List_strings = concatenate_string(words)
len(List_strings)

5171

In [12]:
print List_strings[0]
print List_strings[7]

i finished my courses x4 of ipilimumab went off pembrolizumab and unfortunately have had signifcant growth
there is clear evidence that interferon does have benefits for 3a patients


In [10]:
posts['Posts']= List_strings
posts.to_csv("Corrected_3.csv")

## Step 3 : Training and testing split
For classification model , the required training and testing dataset can be prepared with tts function . 

In [13]:
# Dependent variable is defined
y = posts["Tag"]
subsample_size = 1.0
len(y)

5171

In [46]:
from sklearn.cross_validation import train_test_split as tts
from scipy import sparse

#Training and testing split - with a split of 70:30
X_train, X_test, y_train, y_test = tts(List_strings,y, test_size=0.2, random_state=123)


## Step 4 : Feature Generation
### Vectorizing
For feature generation, we need to create a matrix of vectorized features. For converting a collection of raw documents to a matrix of features, we use TF-IDF vectorizer (Term Frequency – Inverse Document Frequency). It assigns scores/weights to each word, which is considered as a data point for the model.

<b>Term Frequency: This summarizes how often a given word appears within a document.</b><br>
<b>Inverse Document Frequency: This downscales words that appear a lot across documents.</b><br>
TF-IDF are word frequency scores that try to highlight words that are more interesting i.e. frequent in a document, but not across documents.

The Tfidf Vectorizer will tokenize documents, learn the vocabulary and inverse document frequency weightings, and allows to encode new documents.

The dependent variable in the dataset has skewness which has to be eliminated before passing it through the model. 
The appended list of strings is passed through tfidf-vectorizer which a sparse csr matrix  .The generated matrix are used as features for the dataset ,with values being the tfidf score


In [111]:
vectorizer = TfidfVectorizer( analyzer = "word" ,use_idf= True , ngram_range= (1,2), max_features= 20000 )

X_train_feat = vectorizer.fit_transform(X_train)

X_test_feat = vectorizer.transform(X_test)

In [112]:
features = vectorizer.fit_transform(posts['Tokenized.Sentences']).toarray()

In [109]:
tfidf_tags = []
top_n = -5
feature_array = np.array(vectorizer.get_feature_names())
tfidf_sorting = np.argsort(features)
for i, e in enumerate(tfidf_sorting):
    tmp_tags = []
    indexes = e[top_n:]
    for idx in indexes:
        cur_tag = feature_array[idx]
        if features[i][idx] > 0.1 and len(cur_tag)>3 and '_' not in cur_tag:
            tmp_tags.append(cur_tag.replace(' ', '-'))
    tfidf_tags.append(",".join(tmp_tags))

In [110]:
tfidf_tags

[u'unfortunately,courses,off-pembrolizumab,finished-my,went-off',
 u'had-some,but-here,some-success,here-are,some',
 u'just-so,first-or,for-the,go-for',
 u'stuff,treatment-was,was-before,the-news',
 u'tried-ipilimumab,but-only,combination-is,only-to,decided-that',
 u'or-both,or-it,might,might-be,it-might',
 u'rounds,surgeries,had-aldesleukin,lung-and,aldesleukin-rounds',
 u'that-interferon,interferon-does,for-3a,does-have,evidence-that',
 u'against,decided-against,against-interferon,3a-and,and-decided',
 u'light,just-to,sensitive,wear,always',
 u'adjuvant-treatment,in-october,approved-in,was-fda,stage-adjuvant',
 u'results-from,great-results,and-for,your-dad,for-continued',
 u'ipilimumab-and,ipilimumab-ipilimumab,had-ipilimumab,aaron',
 u'started-on,ipilimumab-last,winter,so-started,last-winter',
 u'and-start,reasonable,start-you,would-take,point-and',
 u'what,combination,data,the-data,braf',
 u'risks,benefits-and,patients-as,to-patients,therapy',
 u'your-oncologist,and-your,going-for,

In [113]:
from sklearn.multiclass import OneVsRestClassifier
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english' , use_idf=True )),
    ('clf', OneVsRestClassifier(MultinomialNB(
        fit_prior=True, class_prior=None))),
])
parameters = {
    'tfidf__max_df': (0.25, 0.5, 0.75),
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'clf__estimator__alpha': (1e-2, 1e-3)
}

## Step 5 : Balancing using Synthetic Minority Over-sampling Technique

This technique is followed to avoid overfitting which occurs when exact replicas of minority instances are added to the main dataset. A subset of data is taken from the minority class as an example and then new synthetic similar instances are created. These synthetic instances are then added to the original dataset. The new dataset is used as a sample to train the classification models.

SMOTE is used to create a balanced dataset which can be passed through the best-fit classifier model. The random state or seed should be equal to the assigned for the selected model

Input can be a spicy sparse matrix or numpy array


In [114]:
print "No of 1:",len(np.where(y_train==1)[0])
print "No of 0:",len(np.where(y_train==0)[0])

No of 1: 1797
No of 0: 2339


In [79]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=123, ratio = 0.8)
X_res, y_res = sm.fit_sample(X_train_feat,y_train)

C:\Anaconda2\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


In [80]:
print "No of 1:",len(np.where(y_res==1)[0])
print "No of 0:",len(np.where(y_res==0)[0])

No of 1: 1871
No of 0: 2339


### Running Linear SVM model 
Support vector mahcine is a supervised learning method used for classification, regression and outliers detection. It is memory efficient as it uses a subset of training points in the decision function (called support vectors) . It works good for both binary and multi-class classification . It is flexible as we can select custom kernel as per the training data

SVM is independent of dimensionality of feature space. SVM uses overfitting protection , hence it can handle large number of features which are there in text classifiers

Balanced training set is in csr matrix format, which is used as independent variables and dependent variable is an array with equal dimensions for rows to that of training dataset


In [81]:
svm_classifier = SVC(kernel= 'linear' , C =1 )
svm_model = svm_classifier.fit(X_res ,y_res) 
y_pred_svm = svm_model.predict(X_test_feat)

print(clsr(y_test.tolist(), y_pred_svm.tolist()))

             precision    recall  f1-score   support

          0       0.83      0.85      0.84       570
          1       0.81      0.78      0.80       465

avg / total       0.82      0.82      0.82      1035



### Running Naive Bayes model

The multinomial Naive Bayes classifier is suitable for classification with discrete features (e.g., word counts for text classification). The multinomial distribution normally requires integer feature counts. However, in practice, fractional counts such as tf-idf .

In [29]:
NB_classifier = MultinomialNB()
NB_model = NB_classifier.fit(X_res ,y_res) 

y_pred_NB = NB_model.predict(X_test_feat)

print(clsr(y_test.tolist(), y_pred_NB.tolist()))

             precision    recall  f1-score   support

          0       0.80      0.90      0.84       860
          1       0.85      0.72      0.78       692

avg / total       0.82      0.82      0.81      1552



### Running KNearestNeighbour Model

kNN is powerful as it does not assume anything about the data, other than a distance measure can be calculated consistently between any two instances. It is non-parametric or non-linear as it does not assume a functional form.


In [30]:
KNN_classifier = KNeighborsClassifier()
KNN_model = KNN_classifier.fit(X_res ,y_res) 

y_pred_KNN = KNN_model.predict(X_test_feat)


print(clsr(y_test.tolist(), y_pred_KNN.tolist()))

             precision    recall  f1-score   support

          0       0.81      0.14      0.24       860
          1       0.47      0.96      0.63       692

avg / total       0.66      0.51      0.41      1552



### K-Fold Cross Validation 

Comparing the performance of all the 3 classifiers, we find that SVM works better. To make sure that the model is not overfitting the training sample, we run a K-Fold Cross Validation test

K-fold is also a  model validation technique for assessing the effectiveness of the model, whether it is overfitting or underfitting. It is mainly used in settings where the goal is prediction, and one wants to estimate how accurately a predictive model will perform in practice . It randomly splits the entire dataset into k-folds. For each k folds in your dataset ,test the model on k – 1 folds of the data set. 

In [31]:
from sklearn.model_selection import KFold 
kf = KFold(n_splits=4)
for train_index, test_index in kf.split(List_strings):
    
    #Split train data
    string_train, string_test = np.asarray(List_strings)[train_index], np.asarray(List_strings)[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Vectorize the text
    vectorizer_kfold = TfidfVectorizer(use_idf= True)
    X_train = vectorizer_kfold.fit_transform(string_train)
    X_test = vectorizer_kfold.transform(string_test)
    
    # Train the classifier
    classifier = SVC(kernel='linear' ,C = 1)
    model = classifier.fit(X_train ,y_train)
    y_pred = model.predict(X_test)
    
    print(clsr(y_test.tolist(), y_pred.tolist()))

             precision    recall  f1-score   support

          0       0.74      0.91      0.81       648
          1       0.88      0.67      0.76       645

avg / total       0.81      0.79      0.79      1293

             precision    recall  f1-score   support

          0       0.87      0.82      0.84       768
          1       0.76      0.82      0.79       525

avg / total       0.82      0.82      0.82      1293

             precision    recall  f1-score   support

          0       0.89      0.79      0.84       791
          1       0.72      0.85      0.78       502

avg / total       0.83      0.82      0.82      1293

             precision    recall  f1-score   support

          0       0.82      0.87      0.85       702
          1       0.84      0.77      0.80       590

avg / total       0.83      0.83      0.83      1292



## Predict on a test dataset

Comparing the performance of all the models, we see that SVM performs well. So we predict the test dataset using the trained SVM classifier

In [68]:
# Data to be predicted
posts_test = pd.read_csv('DF_100_user_v1_5-16-2018.csv')

Words_test = preprocessing(posts_test['Tokenized.Sentences'])

# Preprocessed data to be predicted
List_strings_test = concatenate_string(Words_test)

# Vectorize the data to be predicted 
X_pred = vectorizer.transform(List_strings_test)

# Predict using the trained SVM classifier
y_pred = svm_model.predict(X_pred)
len(y_pred)

9907

In [69]:
# Making the predicted data set for 100 test users
y_pred = pd.DataFrame(y_pred)
posts_test['New_tag'] = y_pred
posts_test['Preprocessed_Tokenized_Sent'] = List_strings_test
posts_test = posts_test.drop('Tokenized.Sentences',axis=1)

In [70]:
# Removing rows with All NA's. NAs can be created after preprocessing.
posts_test.dropna(axis=0,how='any', inplace=True)
posts_test = posts_test[(posts_test['Preprocessed_Tokenized_Sent'] != '')&(posts_test['Preprocessed_Tokenized_Sent'] != 'nan')]
#Reset index
posts_test = posts_test.reset_index(drop= True)
posts_test.to_csv("DF_100_users_new_tag_5-16-2018.csv" , index = False)

In [72]:
posts_test.head()

Post_id    User            date  forum  User_id  Sent_id  New_tag  \
0       44  Delray  2/9/2016 21:09      3     1084        1        0   
1       44  Delray  2/9/2016 21:09      3     1084        2        1   
2       44  Delray  2/9/2016 21:09      3     1084        3        0   
3       44  Delray  2/9/2016 21:09      3     1084        4        1   
4       44  Delray  2/9/2016 21:09      3     1084        5        1   

                         Preprocessed_Tokenized_Sent  
0  on jan 21 2016 5 40 pm shaney2 wrote hi everyo...  
1  my husband was diagnosed last may with melanom...  
2  the tumor was removed as well as the nearest l...  
3  because of the size and depth of the tumor mor...  
4                      5 weeks of radiation followed

# Next steps

An ensemble technique can be used to improve the accuracy. Using CTAKES and pattern extraction code, numerical features can be engineered which can be trained by another classifier and the outputs from both the classifier can be fed to a logistic regression model. The hypothesis is that, features like number of medication, temporal mentions and other such features will help the classifier predict better

In [17]:
text = "Nivo Nivo Ipi vemura Nivo"

import itertools as it

lt = tokenize(text)
print lt

<generator object tokenize at 0x00000000173C1990>


In [18]:
no_repeat = []

for i in lt:
    print i
    if (len(no_repeat)!= 0 and i!=no_repeat[-1]) or (len(no_repeat)==0):
        no_repeat.append(i)

nivo
nivo
ipi
vemura
nivo


In [19]:
no_repeat

['nivo', 'ipi', 'vemura', 'nivo']